# **PROYEK ANALISIS SENTIMEN: SCRAPING DATA ULASAN APLIKASI MAMIKOS**

- **Nama:** Muhammad Husain Fadhlillah
- **Email Student:** mc006d5y2343@student.devacademy.id
- **Cohort ID:** MC006D5Y2343

Notebook ini bertujuan untuk melakukan scraping data ulasan (reviews) dari aplikasi Mamikos di Google Play Store.

## **Bagian 1: Proses Pengumpulan Data dari Google Play Store**

### 1. Instalasi dan Import Library
Tahap pertama adalah mempersiapkan semua library yang dibutuhkan untuk proses scraping data.

In [1]:
# Instalasi library google-play-scraper
%pip install google-play-scraper

# Import library yang diperlukan
import pandas as pd
import numpy as np
import os
import time
from google_play_scraper import reviews, Sort

# Untuk mengabaikan peringatan
import warnings
warnings.filterwarnings('ignore')

print("Library berhasil diimport.")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Library berhasil diimport.


### 2. Penentuan Parameter Scraping
Menentukan ID aplikasi "Mamikos" dan jumlah data yang ingin diambil untuk setiap kategori skor. Tujuannya adalah untuk mendapatkan dataset yang seimbang (balanced) untuk menghindari bias pada data.

In [2]:
# ID aplikasi Mamikos di Google Play Store
app_id = 'com.git.mami.kos'

# Jumlah ulasan yang akan di-scrape per kategori skor
# Target adalah >10.000 data, maka ambil 2100 per skor (2100 * 5 = 10.500)
count_per_score = 2100

print(f"Target scraping untuk aplikasi: {app_id}")
print(f"Jumlah ulasan per skor: {count_per_score}")

Target scraping untuk aplikasi: com.git.mami.kos
Jumlah ulasan per skor: 2100


### 3. Proses Scraping Data (Balanced Score)
Proses ini akan melakukan iterasi untuk setiap skor (1-5) dan mengambil ulasan sesuai jumlah yang telah ditentukan. Menggunakan fungsi `reviews` yang memungkinkan filter berdasarkan skor.

In [3]:
# List untuk menampung semua hasil scraping
all_reviews_list = []

# Loop untuk setiap skor dari 1 sampai 5
for score in range(1, 6):
    print(f"--- Scraping ulasan untuk skor: {score} ---")
    try:
        # Mengambil ulasan dengan filter skor
        result, _ = reviews(
            app_id,
            lang='id',                # Bahasa Indonesia
            country='id',             # Negara Indonesia
            sort=Sort.MOST_RELEVANT,  # Urutkan berdasarkan relevansi
            count=count_per_score,    # Jumlah ulasan per skor
            filter_score_with=score   # Filter berdasarkan skor
        )

        # Menambahkan informasi skor ke setiap ulasan
        for r in result:
            r['score_category'] = score # Tambahkan kolom baru untuk skor

        all_reviews_list.extend(result)
        print(f"Berhasil mendapatkan {len(result)} ulasan untuk skor {score}.")

    except Exception as e:
        print(f"Terjadi error saat scraping skor {score}: {e}")

    # Beri jeda untuk menghindari blokir dari server
    time.sleep(2)

print(f"\nTotal ulasan yang berhasil di-scrape: {len(all_reviews_list)}")

--- Scraping ulasan untuk skor: 1 ---
Berhasil mendapatkan 1670 ulasan untuk skor 1.
--- Scraping ulasan untuk skor: 2 ---
Berhasil mendapatkan 470 ulasan untuk skor 2.
--- Scraping ulasan untuk skor: 3 ---
Berhasil mendapatkan 791 ulasan untuk skor 3.
--- Scraping ulasan untuk skor: 4 ---
Berhasil mendapatkan 1043 ulasan untuk skor 4.
--- Scraping ulasan untuk skor: 5 ---
Berhasil mendapatkan 2100 ulasan untuk skor 5.

Total ulasan yang berhasil di-scrape: 6074


### 4. Konversi ke DataFrame dan Eksplorasi Awal
Hasil scraping yang berupa list of dictionaries akan diubah menjadi DataFrame Pandas untuk kemudahan analisis. Melakukan pengecekan awal untuk memastikan data sesuai harapan.

In [4]:
# Konversi list ke DataFrame
df_reviews = pd.DataFrame(all_reviews_list)

# Tampilkan informasi dasar DataFrame
print("Informasi DataFrame:")
df_reviews.info()

# Tampilkan 5 baris pertama
print("\nContoh 5 baris pertama data:")
display(df_reviews.head())

# Cek distribusi skor untuk memastikan data seimbang
print("\nDistribusi jumlah ulasan berdasarkan skor:")
print(df_reviews['score'].value_counts())

Informasi DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6074 entries, 0 to 6073
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              6074 non-null   object        
 1   userName              6074 non-null   object        
 2   userImage             6074 non-null   object        
 3   content               6074 non-null   object        
 4   score                 6074 non-null   int64         
 5   thumbsUpCount         6074 non-null   int64         
 6   reviewCreatedVersion  4700 non-null   object        
 7   at                    6074 non-null   datetime64[ns]
 8   replyContent          5535 non-null   object        
 9   repliedAt             5535 non-null   datetime64[ns]
 10  appVersion            4700 non-null   object        
 11  score_category        6074 non-null   int64         
dtypes: datetime64[ns](2), int64(3), object(7)
memory usage:

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,score_category
0,738b1d1d-b23c-4ec8-9f7c-3f5d936924b9,Akun Smurf,https://play-lh.googleusercontent.com/a/ACg8oc...,Cari uang kok dengan memberatkan pemilik kost....,1,32,25.18.0,2025-05-13 14:08:32,Halo bapak/ibu mohon maaf atas ketidaknyamanan...,2025-05-18 13:18:32,25.18.0,1
1,37e14345-895d-4a48-b1c8-a8871c303696,Felix Andri,https://play-lh.googleusercontent.com/a-/ALV-U...,"Ada banyak notif pesan yg perlu dibalas, begit...",1,33,25.13.0,2025-04-05 11:10:11,"Halo, Pak/Bu, Mohon maaf atas ketidaknyamanan ...",2025-02-17 22:36:10,25.13.0,1
2,3c96eaa6-885b-49dc-bc85-d0ff7cb13493,Jhon Connor,https://play-lh.googleusercontent.com/a/ACg8oc...,"Saya udah beli gold plus tetap aja sepi!, belu...",1,17,25.16.0,2025-05-10 18:33:50,"Halo, Pak/Bu mohon maaf atas ketidaknyamanan y...",2025-05-11 00:52:16,25.16.0,1
3,ed9d60da-6341-412b-86f5-a5350675e442,susi suswati wati,https://play-lh.googleusercontent.com/a-/ALV-U...,"aplikasi ini skrg memberatkan pemilik kost,bay...",1,4,25.20.0,2025-05-23 12:37:13,None,NaT,25.20.0,1
4,01e4057c-f189-4ad7-990b-607cfbb10bfd,Andhika Fajar Ismail,https://play-lh.googleusercontent.com/a-/ALV-U...,Setelah taro nomor telpon di mamikos banyak yg...,1,1,25.18.0,2025-05-19 17:20:17,None,NaT,25.18.0,1



Distribusi jumlah ulasan berdasarkan skor:
5    2100
1    1670
4    1043
3     791
2     470
Name: score, dtype: int64


### 5. Penyimpanan Dataset
Dataset yang sudah menjadi DataFrame akan dsimpan ke dalam format file `.csv` untuk digunakan pada tahap selanjutnya (preprocessing dan pemodelan).

In [5]:
# Membuat direktori 'dataset'
if not os.path.exists('dataset'):
    os.makedirs('dataset')

# Menentukan path file
output_path = 'dataset/dataset_mamikos_reviews.csv'

# Menyimpan DataFrame ke file CSV
df_reviews.to_csv(output_path, index=False)

print(f"Dataset berhasil disimpan di: {output_path}")

Dataset berhasil disimpan di: dataset/dataset_mamikos_reviews.csv
